In [ ]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 67.1 MB/s eta 0:00:00


In [ ]:
import pulp
from scipy.spatial.distance import euclidean
import random
import numpy as np

# Definir coordenadas aleatorias para estaciones
coord = {}
for s in S:
    coord[s] = np.array([random.uniform(0, 10), random.uniform(0, 10)])

# Definir distancias entre estaciones
distance = {}
for s1 in S:
    for s2 in S:
        if s1 != s2:
            distance[s1, s2] = euclidean(coord[s1], coord[s2])


In [ ]:
# Conjuntos
D = {1, 2, 3, 4, 5} # Conjunto de puntos de demanda
S = {1, 2, 3, 4, 5} # Conjunto de posibles estaciones
T = {1, 2, 3} # Conjunto de tamaños de estación

# Parámetros
Dj = {1: 10, 2: 15, 3: 5, 4: 20, 5: 8} # Demanda de cada punto de demanda
Tt = {1: 1, 2: 2, 3: 3} # Capacidad de cada tamaño de estación
B = 20 # Presupuesto máximo
D_max = 100
D_min = 50

# Costos
C = {(1, 1): 5, (1, 2): 8, (1, 3): 10,
     (2, 1): 7, (2, 2): 11, (2, 3): 13,
     (3, 1): 6, (3, 2): 9, (3, 3): 12,
     (4, 1): 8, (4, 2): 12, (4, 3): 15,
     (5, 1): 5, (5, 2): 7, (5, 3): 9}

# Distancias
r = {(1, 1): 0, (1, 2): 1, (1, 3): 1, (1, 4): 2, (1, 5): 1,
     (2, 1): 1, (2, 2): 0, (2, 3): 2, (2, 4): 1, (2, 5): 1,
     (3, 1): 1, (3, 2): 2, (3, 3): 0, (3, 4): 1, (3, 5): 1,
     (4, 1): 2, (4, 2): 1, (4, 3): 1, (4, 4): 0, (4, 5): 2,
     (5, 1): 1, (5, 2): 1, (5, 3): 1, (5, 4): 2, (5, 5): 0}

In [ ]:
# Variables de decisión
x = pulp.LpVariable.dicts('x', [(s, t) for s in S for t in T], lowBound=0, upBound=1, cat=pulp.LpInteger)
y = pulp.LpVariable.dicts('y', [(s, j) for s in S for j in D], lowBound=0, upBound=1, cat=pulp.LpInteger)


In [ ]:
# Creación del problema
problem = pulp.LpProblem('Demand Coverage', pulp.LpMaximize)

# Función objetivo
problem += pulp.lpSum(Dj[j] * pulp.lpSum(y[s, j] for s in S for t in T) for j in D)

/usr/local/lib/python3.8/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
# Restricciones
# Presupuesto
problem += pulp.lpSum(C[s, t] * x[s, t] for s in S for t in T) <= B

# Asignación
for j in D:
  for s in S:
    if r[s, j] > 0:
      problem += y[s, j] <= pulp.lpSum(x[ss, tt] for ss in S for tt in T if r[s, ss] <= r[ss, j])

#Capacidad
for s in S:
  problem += pulp.lpSum(Tt[t] * x[s, t] for t in T) >= pulp.lpSum(Dj[j] * y[s, j] for j in D)

# Restricción de distancia entre estaciones
for s1 in S:
    for s2 in S:
        if s1 != s2:
            for t1 in T:
                for t2 in T:
                    if t1 <= t2:
                        d = distance[s1, s2]
                        if d > D_max:
                            problem += x[s1, t1] + x[s2, t2] <= 1
                        elif d <= D_min:
                            problem += x[s1, t1] + x[s2, t2] <= 2
                        else:
                            problem += x[s1, t1] + x[s2, t2] - x[s1, t1]*x[s2, t2] <= 1


In [ ]:
#Solución
problem.solve()

#Imprimir resultados
print(f'Estado de la solución: {pulp.LpStatus[problem.status]}')
print(f'Valor objetivo: {pulp.value(problem.objective)}')

print('Estaciones abiertas:')
for s in S:
  for t in T:
    if x[s, t].value() > 0:
      print(f'Estación {s} de tamaño {t}')

print('Asignaciones de estaciones a puntos de demanda:')
for s in S:
  for j in D:
    if y[s, j].value() > 0:
      print(f'Estación {s} asignada al punto de demanda {j}')

Estado de la solución: Optimal
Valor objetivo: 15.0
Estaciones abiertas:
Estación 1 de tamaño 2
Estación 1 de tamaño 3
Asignaciones de estaciones a puntos de demanda:
Estación 1 asignada al punto de demanda 3
